# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,279.614240,-0.147413,1234,279.466827,279.761653,558.933655,559.523306
6,634.265880,0.443675,1234,634.709555,633.822205,1269.419110,1267.644409
7,233.754157,0.982017,1234,234.736174,232.772140,469.472347,465.544280
8,405.665450,-0.923127,1234,404.742323,406.588577,809.484647,813.177155
9,10.706575,-0.412627,1234,10.293948,11.119201,20.587896,22.238403
10,543.393956,0.443011,1234,543.836967,542.950944,1087.673934,1085.901889
11,803.152052,0.577118,1234,803.729170,802.574934,1607.458339,1605.149869
12,686.296289,-0.365139,1234,685.931150,686.661428,1371.862299,1373.322856
13,364.344271,-0.653601,1234,363.690670,364.997873,727.381340,729.995745
14,355.594797,-0.610052,1234,354.984745,356.204849,709.969489,712.409698


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-279.466827,279.761653
6,-634.709555,633.822205
7,-234.736174,232.772140
8,-404.742323,406.588577
9,-10.293948,11.119201
10,-543.836967,542.950944
11,-803.729170,802.574934
12,-685.931150,686.661428
13,-363.690670,364.997873
14,-354.984745,356.204849


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-279.466827,279.761653
6,-634.709555,633.822205
7,-234.736174,232.772140
8,-404.742323,406.588577
9,-10.293948,11.119201
10,-543.836967,542.950944
11,-803.729170,802.574934
12,-685.931150,686.661428
13,-363.690670,364.997873
14,-354.984745,356.204849


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-279.46682733,  279.76165314],
       [-634.70955506,  633.82220456],
       [-234.73617374,  232.77214001],
       [-404.74232331,  406.58857732],
       [ -10.29394823,   11.11920143],
       [-543.8369671 ,  542.95094426],
       [-803.72916956,  802.57493439],
       [-685.93114973,  686.66142799],
       [-363.69066994,  364.99787273],
       [-354.98474456,  356.20484893]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,556.933655,-559.523306
6,1267.419110,-1267.644409
7,467.472347,-465.544280
8,807.484647,-813.177155
9,18.587896,-22.238403
10,1085.673934,-1085.901889
11,1605.458339,-1605.149869
12,1369.862299,-1373.322856
13,725.381340,-729.995745
14,707.969489,-712.409698


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.279614,0.426294
6,0.634266,0.721838
7,0.233754,0.991008
8,0.405665,0.038436
9,0.010707,0.293687
10,0.543394,0.721506
11,0.803152,0.788559
12,0.686296,0.317430
13,0.364344,0.173199
14,0.355595,0.194974


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,279.614240,-0.147413
6,634.265880,0.443675
7,233.754157,0.982017
8,405.665450,-0.923127
9,10.706575,-0.412627
10,543.393956,0.443011
11,803.152052,0.577118
12,686.296289,-0.365139
13,364.344271,-0.653601
14,355.594797,-0.610052


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
